In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE169149"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE169149"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE169149.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE169149.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE169149.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Evaluation of tofacitinib in cutaneous sarcoidosis"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['subject status: Sarcoidosis patient', 'subject status: healthy control'], 1: ['treatment: none', 'treatment: tofacitinib'], 2: ['tissue: Blood']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check the available data in the sample characteristics dictionary
print("Sample Characteristics Dictionary:")
print({0: ['subject status: Sarcoidosis patient', 'subject status: healthy control'], 
       1: ['treatment: none', 'treatment: tofacitinib'], 
       2: ['tissue: Blood']})

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a blood gene expression dataset for sarcoidosis
is_gene_available = True  # Blood tissue samples likely contain gene expression data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 0  # The trait (Allergies/Sarcoidosis status) is in row 0
age_row = None  # Age information is not available in the sample characteristics
gender_row = None  # Gender information is not available in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait data to binary format (0: control, 1: sarcoidosis)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "sarcoidosis" in value.lower() or "patient" in value.lower():
        return 1
    elif "healthy" in value.lower() or "control" in value.lower():
        return 0
    return None

# Define convert_age and convert_gender as None since the data is not available
convert_age = None
convert_gender = None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Load clinical data (this variable should be provided from previous steps)
    # For this example, let's assume clinical_data is already defined
    
    # Make sure the clinical_data variable exists before using it
    try:
        # Extract clinical features using the library function
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical data
        print("Preview of extracted clinical data:")
        print(preview_df(selected_clinical_df))
        
        # Save the clinical data to a CSV file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Clinical data not available from previous steps")


Sample Characteristics Dictionary:
{0: ['subject status: Sarcoidosis patient', 'subject status: healthy control'], 1: ['treatment: none', 'treatment: tofacitinib'], 2: ['tissue: Blood']}
Preview of extracted clinical data:
{'GSM5176932': [1.0], 'GSM5176933': [1.0], 'GSM5176934': [1.0], 'GSM5176935': [1.0], 'GSM5176936': [1.0], 'GSM5176937': [1.0], 'GSM5176938': [1.0], 'GSM5176939': [1.0], 'GSM5176940': [1.0], 'GSM5176941': [1.0], 'GSM5176942': [1.0], 'GSM5176943': [1.0], 'GSM5176944': [1.0], 'GSM5176945': [1.0], 'GSM5176946': [1.0], 'GSM5176947': [1.0], 'GSM5176948': [1.0], 'GSM5176949': [1.0], 'GSM5176950': [1.0], 'GSM5176951': [1.0], 'GSM5176952': [0.0], 'GSM5176953': [0.0], 'GSM5176954': [0.0], 'GSM5176955': [0.0], 'GSM5176956': [0.0], 'GSM5176957': [0.0], 'GSM5176958': [0.0], 'GSM5176959': [0.0], 'GSM5176960': [0.0], 'GSM5176961': [0.0], 'GSM5176962': [0.0]}
Clinical data saved to ../../output/preprocess/Allergies/clinical_data/GSE169149.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the identifiers provided

# The observed identifiers are numeric (1, 2, 3, etc.) which are not standard human gene symbols
# Standard human gene symbols would typically be alphanumeric strings like "BRCA1", "TP53", etc.
# These appear to be just row indices or probe IDs that would need to be mapped to actual gene symbols

# Therefore, gene mapping is required
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'Assay': ['AARSD1', 'ABHD14B', 'ABL1', 'ACAA1', 'ACAN'], 'OlinkID': ['OID21311', 'OID20921', 'OID21280', 'OID21269', 'OID20159'], 'PT_ACC': ['Q9BTE6', 'Q96IU4', 'P00519', 'P09110', 'P16112'], 'Panel': ['Oncology', 'Neurology', 'Oncology', 'Oncology', 'Cardiometabolic'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers and gene annotation data
# From the output in steps 3 and 5, we can see:
# - Gene identifiers in gene expression data are numeric strings ('1', '2', '3', etc.)
# - In the gene annotation, the 'ID' column matches these identifiers, and 'Assay' column contains gene symbols

# 2. Get a gene mapping dataframe by extracting the relevant columns
prob_col = 'ID'  # Column containing probe identifiers
gene_col = 'Assay'  # Column containing gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few rows of the resulting gene expression dataframe to verify the mapping
print("First few rows of gene-level expression data:")
print(gene_data.head())


First few rows of gene-level expression data:
         GSM5176932  GSM5176933  GSM5176934  GSM5176935  GSM5176936  \
Gene                                                                  
AARSD1       3.4878      3.6728      4.1162      4.7169      3.6683   
ABHD14B      1.7953      1.6497      3.0499      2.6048      1.9029   
ABL1         2.6829      2.4827      3.3944      3.3331      2.6946   
ACAA1        1.4306      0.9938      3.3866      2.7677      3.3732   
ACAN         0.3385      0.2088      0.0150     -0.4124     -0.6523   

         GSM5176937  GSM5176938  GSM5176939  GSM5176940  GSM5176941  ...  \
Gene                                                                 ...   
AARSD1       3.6745      5.1706      3.0317      3.1368      4.8808  ...   
ABHD14B      1.4334      3.4131      2.1466      1.4771      4.1245  ...   
ABL1         3.1111      5.3688      2.6608      1.5761      4.6803  ...   
ACAA1        2.4944      3.2448      2.1226      0.4455      3.5292  ...   


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...


Gene data shape after normalization: (1453, 31)
Normalized gene data saved to ../../output/preprocess/Allergies/gene_data/GSE169149.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM5176932': [1.0], 'GSM5176933': [1.0], 'GSM5176934': [1.0], 'GSM5176935': [1.0], 'GSM5176936': [1.0], 'GSM5176937': [1.0], 'GSM5176938': [1.0], 'GSM5176939': [1.0], 'GSM5176940': [1.0], 'GSM5176941': [1.0], 'GSM5176942': [1.0], 'GSM5176943': [1.0], 'GSM5176944': [1.0], 'GSM5176945': [1.0], 'GSM5176946': [1.0], 'GSM5176947': [1.0], 'GSM5176948': [1.0], 'GSM5176949': [1.0], 'GSM5176950': [1.0], 'GSM5176951': [1.0], 'GSM5176952': [0.0], 'GSM5176953': [0.0], 'GSM5176954': [0.0], 'GSM5176955': [0.0], 'GSM5176956': [0.0], 'GSM5176957': [0.0], 'GSM5176958': [0.0], 'GSM5176959': [0.0], 'GSM5176960': [0.0], 'GSM5176961': [0.0], 'GSM5176962': [0.0]}
Clinical data saved to ../../output/preprocess/Allergies/clinical_data/GSE169149.csv
Linking clinical and genetic data.

Linked data shape after handling missing values: (31, 1454)
Checking for bias in trait distribution...
For the feature 'Allergies', the least common label is '0.0' with 11 occurrences. This represents 35.48% of the dataset.
The distribution of the feature 'Allergies' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Allergies/cohort_info.json
Dataset usability: True
Linked data saved to ../../output/preprocess/Allergies/GSE169149.csv
